<a href="https://colab.research.google.com/github/LinusBach/SentimentAnalysis/blob/main/sentiAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple sentiment analysis

Sentiment analysis, using iMDB database

First, implement and train a feedforward NN model with TF-IDF. And then train your
model using word2vec embedding. Report both training and development accuracy on
the dataset. Try to use stochastic gradient descent or (mini-batch) stochastic gradient
descent!

In [ ]:
!pip install tensorflow
!pip install nltk
!pip install gensim

### imports


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow import keras
import nltk
import csv
import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
import gensim

### load dataset into memory
return a list of docs and a list of respective labels

In [ ]:
def load_data (filename):
  content = list()
  labels = list()

  has_header = True
  # detect if file has a header
  # with open(filename, 'r') as file:
  #   sample = file.read(64)
  #   has_header = csv.Sniffer().has_header(sample)

  with open(filename, 'r') as file:
    reader = csv.reader(file)
    # skip first line if file has a header 
    if has_header:
      next(reader)
    for c, l in reader:
      content.append(c)
      labels.append(l)
  return content, labels

### turn a dataset into clean tokens

In [ ]:
def clean_data(data):
  corpus = list()
  corp_voc = dict()
  stop = nltk.corpus.stopwords.words("english")
  # regex tokenizer, find words, numbers and words containing '
  tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+(?:'\w)?")
  for doc in data:
    doc = tokenizer.tokenize(doc)
    doc_cleaned = dict()
    for tok in doc:
      # make all words lower case
      tok = tok.lower()
      # filter out numbers 
      if not tok.isdigit() and tok not in stop:
        # add clean token to document
        if tok in doc_cleaned:
          doc_cleaned[tok] += 1
        else:
          doc_cleaned[tok] = 1
    for tok in doc_cleaned.keys():
    # increase corpus vocabulary
        if tok in corp_voc:
          corp_voc[tok] += 1
        else:
          corp_voc[tok] = 1
    corpus.append(doc_cleaned)
  return corpus, corp_voc

# filter all words out of a corpus that are not in a vocabulary
def get_filtered_corpus(corpus, vocab):
  clean_corpus = list()
  for doc in corpus:
    clean_doc = dict()
    for tok in doc:
      if tok in vocab:
        clean_doc[tok] = doc[tok]
    clean_corpus.append(clean_doc)
  return clean_corpus

### preprocess the dataset

tf-idf vectorizer

In [ ]:

# corpus must be a list of dicts of form (token: occurences)
# vocab must be a dict of form (token: documents in corpus containing token)
def preprocess_tf_idf(corpus, vocab):
  processed = np.zeros((len(corpus), len(vocab)))
  idf = get_idf(vocab)
  token_order = {tok: i for i, tok, in enumerate(sorted(vocab.keys()))}
  for n_doc, doc in enumerate(corpus):
    tf = get_tf(doc)
    for tok in set(doc):
      tok_pos = token_order[tok]
      processed[n_doc][tok_pos] = tf[tok]*idf[tok]
  return processed

def get_tf(doc):
  tf = dict()
  for tok, occ in doc.items():
    tf[tok] = occ / len(doc)
  return tf

def get_idf(corp_voc):
  idf = dict()
  for tok, docs_containing in corp_voc.items():
    idf[tok] = np.log10(len(corp_voc) / docs_containing)
  return idf


semantic embeddings
pooling word2vec embeddings

In [ ]:
def preprocess_sem_vec(corpus):
  processed = list()
  model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
  for n_doc, doc in enumerate(corpus):
    vecs = [occ*model[tok] for tok, occ in doc.items() if tok in model.key_to_index]
    processed.append(np.mean(vecs, axis=0))
  return np.array(processed)

### define the model


In [ ]:
def define_model(input_dim):
  model = keras.models.Sequential()
  model.add(layers.Dense(511, input_dim=input_dim, activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(255, activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(127, activation='relu'))
  model.add(layers.Dense(2, activation='softmax'))
  return model

### classify a review as negative or positive.

In [ ]:
def predict_sentiment(model, doc):
  return model.predict(doc)

## Running the model

Setting constants

In [75]:
LEARNING_RATE = 0.001
BATCH_SIZE = 32
EPOCHS = 20
# minimum ocuurences for a word to be regarded
VOCAB_SIZE = 20000
# number of most frequent words to be disregarded
HIGHER_CUTOFF = 0

load training data

In [ ]:
# read data from file
raw_data, labels = load_data("Train.csv")
full_corpus, full_vocab = clean_data(raw_data)
# select part of vocabulary
frequencies = sorted(full_vocab.items(), key=lambda x : x[1], reverse=True)
vocab = {x[0] : x[1] for x in frequencies[HIGHER_CUTOFF:HIGHER_CUTOFF+VOCAB_SIZE]}
corpus = get_filtered_corpus(full_corpus, vocab.keys())
# process labels
one_hot_labels = keras.utils.to_categorical(labels, num_classes=2)

load validation data

In [ ]:
v_raw, v_labels = load_data("Valid.csv")
full_valid_corpus, _ = clean_data(v_raw)
valid_corpus = get_filtered_corpus(full_valid_corpus, vocab)
valid_labels = keras.utils.to_categorical(v_labels, num_classes=2)

load testing data

In [ ]:
t_raw, t_labels = load_data("Test.csv")
full_test_corpus, _ = clean_data(t_raw)
test_corpus = get_filtered_corpus(full_test_corpus, vocab)
test_labels = keras.utils.to_categorical(t_labels, num_classes=2)

train a model on tf-idf data 

In [ ]:
# get tf-idf representation of data
tfidf_data = preprocess_tf_idf(corpus, vocab)
test_data = preprocess_tf_idf(test_corpus, vocab)
valid_data = preprocess_tf_idf(valid_corpus, vocab)

In [ ]:
# build model
tfidf_model = define_model(VOCAB_SIZE,)
tfidf_model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=['accuracy'])
# train model
print("begin training")
tfidf_model.fit(
    tfidf_data,
    one_hot_labels,
    validation_data=(valid_data, valid_labels),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE)
print("begin evaluation")
tfidf_model.evaluate(test_data, test_labels)

begin training
Epoch 1/20
1250/1250 [==============================] - 59s 47ms/step - loss: 0.3144 - accuracy: 0.8626 - val_loss: 0.2616 - val_accuracy: 0.8958
Epoch 2/20
1250/1250 [==============================] - 59s 48ms/step - loss: 0.1471 - accuracy: 0.9450 - val_loss: 0.3074 - val_accuracy: 0.8866
Epoch 3/20
1250/1250 [==============================] - 57s 46ms/step - loss: 0.0673 - accuracy: 0.9733 - val_loss: 0.4385 - val_accuracy: 0.8744
Epoch 4/20
1250/1250 [==============================] - 58s 46ms/step - loss: 0.0248 - accuracy: 0.9896 - val_loss: 0.7215 - val_accuracy: 0.8822
Epoch 5/20
1250/1250 [==============================] - 58s 46ms/step - loss: 0.0125 - accuracy: 0.9959 - val_loss: 0.9748 - val_accuracy: 0.8822
Epoch 6/20
 565/1250 [============>.................] - ETA: 30s - loss: 0.0075 - accuracy: 0.9980

train a model on word2vec embeddings 

In [76]:
# get semantic embeddings of data
w2v_train_data = preprocess_sem_vec(corpus)
w2v_valid_data = preprocess_sem_vec(valid_corpus)
w2v_test_data = preprocess_sem_vec(test_corpus)

In [78]:
# build model
w2v_model = define_model(300,)
w2v_model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=['accuracy'])
# train model
print("begin training")
w2v_model.fit(
    w2v_train_data,
    one_hot_labels,
    validation_data=(w2v_valid_data, valid_labels),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE)
print("begin evaluation")
w2v_model.evaluate(w2v_test_data, test_labels)

begin training
Epoch 1/20
1250/1250 [==============================] - 4s 3ms/step - loss: 0.3884 - accuracy: 0.8260 - val_loss: 0.3462 - val_accuracy: 0.8456
Epoch 2/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.3511 - accuracy: 0.8473 - val_loss: 0.3395 - val_accuracy: 0.8534
Epoch 3/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.3413 - accuracy: 0.8541 - val_loss: 0.3236 - val_accuracy: 0.8624
Epoch 4/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.3377 - accuracy: 0.8549 - val_loss: 0.3246 - val_accuracy: 0.8616
Epoch 5/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.3319 - accuracy: 0.8565 - val_loss: 0.3191 - val_accuracy: 0.8612
Epoch 6/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.3266 - accuracy: 0.8593 - val_loss: 0.3344 - val_accuracy: 0.8596
Epoch 7/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.3230 - accuracy: 0.8619 - val_loss: 0.3362 

[0.34957972168922424, 0.8497999906539917]